<a href="https://colab.research.google.com/github/asheta66/Machine-Learning-2024/blob/main/ELM/PSO_FFNN_ELM_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
# pso_ffnn_vs_elm.py
# ------------------------------------------------------------
# Fair PSO comparison:
#   - PSO-FFNN: optimize ALL weights & biases (1 hidden layer)
#   - PSO-ELM : optimize ONLY input->hidden weights & biases (output = ridge)
# Data:
#   - CSV: air_quality_o3.csv
#   - Inputs: all columns except last
#   - Target: last column
# Figures:
#   - Actual vs Estimated (Train/Test) for both models
#   - PSO Convergence curves (FFNN vs ELM) in 1x2 subplots
# ------------------------------------------------------------

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Tuple, Dict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# =========================
# 0) User-config (ONE place)
# =========================
CSV_PATH             = "air_quality_o3.csv"  # <--- set once
RANDOM_STATE  = 23
TEST_SIZE            = 0.3

# Model structure (shared and FIXED for fairness)
HIDDEN_NEURONS   = 64                     # <--- change if you want a different width
ACTIVATION       = "relu"                 # "relu" | "tanh" | "sigmoid" | "linear"

# ELM ridge regularization on output weights (in scaled target space)
ELM_RIDGE_ALPHA  = 1e-2

# PSO settings (used for BOTH models)
PSO_PARTICLES    = 200
PSO_ITERS        = 250
PSO_W            = 0.72                   # inertia
PSO_C1           = 1.49                   # cognitive
PSO_C2           = 1.49                   # social
PSO_POS_BOUND    = 1.0                    # weights/bias search bounds: [-B, B]
PSO_VEL_BOUND    = 0.3                    # velocity clamp ±

np.random.seed(RANDOM_STATE)

# =========================
# 1) Load CSV & auto-detect X, y
# =========================
df = pd.read_csv(CSV_PATH)
df.columns = [c.strip() for c in df.columns]
assert df.shape[1] >= 2, "CSV must have at least 2 columns."

all_cols   = df.columns.tolist()
target_col = all_cols[-1]        # last column = target
feature_cols = all_cols[:-1]     # the rest = inputs

print(f"Detected target: {target_col}")
print(f"Detected features ({len(feature_cols)}): {feature_cols}")

X_raw = df[feature_cols].copy()
y_raw = df[target_col].astype(float).to_numpy()

# =========================
# 2) Train/Test split + preprocessing
# =========================
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X_raw, y_raw, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

# numeric-only pipeline (median impute + standardize)
x_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
X_train = x_pipe.fit_transform(X_train_raw)
X_test  = x_pipe.transform(X_test_raw)

# Scale target for optimization stability (both models use scaled y during PSO)
y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train_raw.reshape(-1,1)).ravel()
y_test  = y_scaler.transform(y_test_raw.reshape(-1,1)).ravel()

n_samples, n_features = X_train.shape
print(f"Shapes -> X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}")

# =========================
# 3) Utilities
# =========================
def activation_forward(Z: np.ndarray, kind: str) -> np.ndarray:
    if kind == "relu":
        return np.maximum(0.0, Z)
    if kind == "tanh":
        return np.tanh(Z)
    if kind == "sigmoid":
        return 1.0 / (1.0 + np.exp(-Z))
    if kind == "linear":
        return Z
    raise ValueError(f"Unknown activation: {kind}")

def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def regression_report(y_true_real, y_pred_real) -> Dict[str, float]:
    mae  = mean_absolute_error(y_true_real, y_pred_real)
    rmse_v = rmse(y_true_real, y_pred_real)
    r2   = r2_score(y_true_real, y_pred_real)
    mape = float(np.mean(np.abs((y_true_real - y_pred_real) /
                                np.clip(np.abs(y_true_real), 1e-8, None))) * 100.0)
    return {"MAE": mae, "RMSE": rmse_v, "R2": r2, "MAPE_%": mape}

# =========================
# 4) Parameter encodings
# =========================
# FFNN (1 hidden layer, 1 output): params = W1 (d*H) + b1 (H) + W2 (H*1) + b2 (1)
def ffnn_param_size(d: int, H: int) -> int:
    return d*H + H + H + 1

def unpack_ffnn_params(theta: np.ndarray, d: int, H: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float]:
    idx = 0
    W1 = theta[idx: idx + d*H].reshape(d, H); idx += d*H
    b1 = theta[idx: idx + H];                 idx += H
    W2 = theta[idx: idx + H].reshape(H, 1);   idx += H
    b2 = theta[idx];                          idx += 1  # FIX: direct scalar, avoids deprecation
    return W1, b1, W2, b2

def ffnn_forward(X: np.ndarray, theta: np.ndarray, act: str, d: int, H: int) -> np.ndarray:
    W1, b1, W2, b2 = unpack_ffnn_params(theta, d, H)
    Hx = activation_forward(X @ W1 + b1, act)     # (n, H)
    y  = Hx @ W2 + b2                              # (n, 1)
    return y.ravel()

# ELM (1 hidden layer): PSO optimizes W1 (d*H) + b1 (H); output is ridge-closed-form
def elm_param_size(d: int, H: int) -> int:
    return d*H + H

def unpack_elm_params(theta: np.ndarray, d: int, H: int) -> Tuple[np.ndarray, np.ndarray]:
    W1 = theta[:d*H].reshape(d, H)
    b1 = theta[d*H:d*H+H]
    return W1, b1

def elm_fit_predict(X_train: np.ndarray, y_train: np.ndarray,
                    X_eval: np.ndarray, theta: np.ndarray,
                    act: str, d: int, H: int, alpha: float) -> np.ndarray:
    """Return predictions on X_eval (all in scaled target space)."""
    W1, b1 = unpack_elm_params(theta, d, H)
    H_tr = activation_forward(X_train @ W1 + b1, act)  # (n, H)

    # Ridge on augmented H (learns output weights and bias)
    H_tr_aug = np.hstack([H_tr, np.ones((H_tr.shape[0], 1))])  # (n, H+1)
    I = np.eye(H_tr_aug.shape[1])
    beta = np.linalg.solve(H_tr_aug.T @ H_tr_aug + alpha * I, H_tr_aug.T @ y_train)  # (H+1,)

    # Predict
    H_ev = activation_forward(X_eval @ W1 + b1, act)
    H_ev_aug = np.hstack([H_ev, np.ones((H_ev.shape[0], 1))])
    y_hat = H_ev_aug @ beta
    return y_hat.ravel()

# =========================
# 5) PSO (global-best, returns convergence history)
# =========================
def pso_optimize(
    loss_fn, dim: int, swarm_size: int = PSO_PARTICLES, iters: int = PSO_ITERS,
    w: float = PSO_W, c1: float = PSO_C1, c2: float = PSO_C2,
    pos_bound: float = PSO_POS_BOUND, vel_bound: float = PSO_VEL_BOUND,
    seed: int = RANDOM_STATE
):
    rng = np.random.default_rng(seed)
    # Initialize
    Xp = rng.uniform(-pos_bound, pos_bound, size=(swarm_size, dim))
    V  = rng.uniform(-vel_bound, vel_bound, size=(swarm_size, dim))

    # Personal bests
    pbest_pos = Xp.copy()
    pbest_val = np.array([loss_fn(x) for x in Xp])

    # Global best
    g_idx = int(np.argmin(pbest_val))
    gbest_pos = pbest_pos[g_idx].copy()
    gbest_val = float(pbest_val[g_idx])

    # Keep history of best loss
    best_history = [gbest_val]

    for it in range(iters):
        rp, rg = rng.random((swarm_size, dim)), rng.random((swarm_size, dim))
        V = w*V + c1*rp*(pbest_pos - Xp) + c2*rg*(gbest_pos - Xp)
        V = np.clip(V, -vel_bound, vel_bound)
        Xp = Xp + V
        Xp = np.clip(Xp, -pos_bound, pos_bound)

        vals = np.array([loss_fn(x) for x in Xp])

        improved = vals < pbest_val
        pbest_pos[improved] = Xp[improved]
        pbest_val[improved] = vals[improved]

        g_idx = int(np.argmin(pbest_val))
        if pbest_val[g_idx] < gbest_val:
            gbest_val = float(pbest_val[g_idx])
            gbest_pos = pbest_pos[g_idx].copy()

        best_history.append(gbest_val)

        if (it+1) % 20 == 0:
            print(f"[PSO] iter {it+1}/{iters}  best_loss={gbest_val:.5f}")

    return gbest_pos, gbest_val, np.array(best_history)

# =========================
# 6) Define losses for PSO (Train RMSE on scaled y)
# =========================

from sklearn.metrics import mean_squared_error  # already imported above

import numpy as np
EPS = 1e-12  # avoids log(0)

def ffnn_train_loss(theta: np.ndarray) -> float:
    y_hat = ffnn_forward(X_train, theta, ACTIVATION, n_features, HIDDEN_NEURONS)
    mse = mean_squared_error(y_train, y_hat)
    return float(np.log(mse + EPS))   # log-MSE

def elm_train_loss(theta: np.ndarray) -> float:
    y_hat = elm_fit_predict(X_train, y_train, X_train, theta,
                            ACTIVATION, n_features, HIDDEN_NEURONS, ELM_RIDGE_ALPHA)
    mse = mean_squared_error(y_train, y_hat)
    return float(np.log(mse + EPS))   # log-MSE


# --- replace these two functions in Section 6 ---
def ffnn_train_loss(theta: np.ndarray) -> float:
    y_hat = ffnn_forward(X_train, theta, ACTIVATION, n_features, HIDDEN_NEURONS)
    return float(mean_squared_error(y_train, y_hat))  # MSE instead of RMSE

def elm_train_loss(theta: np.ndarray) -> float:
    y_hat = elm_fit_predict(X_train, y_train, X_train, theta,
                            ACTIVATION, n_features, HIDDEN_NEURONS, ELM_RIDGE_ALPHA)
    return float(mean_squared_error(y_train, y_hat))  # MSE instead of RMSE

# =========================
# 7) Run PSO for BOTH models (same PSO settings)
# =========================
print("\n=== Optimizing FFNN with PSO (all weights/biases) ===")
best_ffnn_theta, best_ffnn_loss, ffnn_hist = pso_optimize(ffnn_train_loss, dim=FFNN_DIM)

print("\n=== Optimizing ELM with PSO (hidden weights/biases only) ===")
best_elm_theta, best_elm_loss, elm_hist = pso_optimize(elm_train_loss, dim=ELM_DIM)

# =========================
# 8) Final predictions (convert back to ORIGINAL target scale)
# =========================
# FFNN preds (scaled), then inverse-transform to original scale
ytr_ffnn_scaled = ffnn_forward(X_train, best_ffnn_theta, ACTIVATION, n_features, HIDDEN_NEURONS)
yte_ffnn_scaled = ffnn_forward(X_test,  best_ffnn_theta, ACTIVATION, n_features, HIDDEN_NEURONS)
ytr_ffnn_pred = y_scaler.inverse_transform(ytr_ffnn_scaled.reshape(-1,1)).ravel()
yte_ffnn_pred = y_scaler.inverse_transform(yte_ffnn_scaled.reshape(-1,1)).ravel()

# ELM preds (scaled), then inverse-transform to original scale
ytr_elm_scaled = elm_fit_predict(X_train, y_train, X_train, best_elm_theta,
                                 ACTIVATION, n_features, HIDDEN_NEURONS, ELM_RIDGE_ALPHA)
yte_elm_scaled = elm_fit_predict(X_train, y_train, X_test, best_elm_theta,
                                 ACTIVATION, n_features, HIDDEN_NEURONS, ELM_RIDGE_ALPHA)
ytr_elm_pred = y_scaler.inverse_transform(ytr_elm_scaled.reshape(-1,1)).ravel()
yte_elm_pred = y_scaler.inverse_transform(yte_elm_scaled.reshape(-1,1)).ravel()

# Metrics on original scale
ffnn_train = regression_report(y_train_raw, ytr_ffnn_pred)
ffnn_test  = regression_report(y_test_raw,  yte_ffnn_pred)
elm_train  = regression_report(y_train_raw, ytr_elm_pred)
elm_test   = regression_report(y_test_raw,  yte_elm_pred)

Detected target: O3
Detected features (10): ['AMP_TMP', 'CO', 'NO', 'NO2', 'Nox', 'RH', 'SO2', 'WD', 'WS', 'PM10']
Shapes -> X_train: (766, 10), X_test: (329, 10), y_train: (766,)

=== Optimizing FFNN with PSO (all weights/biases) ===
[PSO] iter 20/250  best_loss=0.55162
[PSO] iter 40/250  best_loss=0.39677
[PSO] iter 60/250  best_loss=0.35688
[PSO] iter 80/250  best_loss=0.33621
[PSO] iter 100/250  best_loss=0.32593
[PSO] iter 120/250  best_loss=0.30508
[PSO] iter 140/250  best_loss=0.29503
[PSO] iter 160/250  best_loss=0.28223
[PSO] iter 180/250  best_loss=0.26376
[PSO] iter 200/250  best_loss=0.25435
[PSO] iter 220/250  best_loss=0.25034
[PSO] iter 240/250  best_loss=0.23962

=== Optimizing ELM with PSO (hidden weights/biases only) ===
[PSO] iter 20/250  best_loss=0.18039
[PSO] iter 40/250  best_loss=0.16303
[PSO] iter 60/250  best_loss=0.14984
[PSO] iter 80/250  best_loss=0.14214
[PSO] iter 100/250  best_loss=0.13732
[PSO] iter 120/250  best_loss=0.12806
[PSO] iter 140/250  best_lo

In [79]:
# =========================
# 9) Figures: Actual vs Estimated (both Train & Test)
# =========================
os.makedirs("figs", exist_ok=True)

def plot_actual_vs_pred_dual(ytr, ytr_pred, yte, yte_pred, model_name, path):
    fig, axes = plt.subplots(2, 1, figsize=(12, 4), sharey=True)

    axes[0].plot(np.asarray(ytr), label="Actual O3")
    axes[0].plot(np.asarray(ytr_pred), label="Estimated O3", alpha=0.9)
    axes[0].set_title(f"{model_name} — Train")
    axes[0].set_xlabel("Samples (order in file)")
    axes[0].set_ylabel("O3")
    axes[0].grid(True, linestyle="--", linewidth=0.5)
    axes[0].legend()

    axes[1].plot(np.asarray(yte), label="Actual O3")
    axes[1].plot(np.asarray(yte_pred), label="Estimated O3", alpha=0.9)
    axes[1].set_title(f"{model_name} — Test")
    axes[1].set_xlabel("Samples (order in file)")
    axes[1].grid(True, linestyle="--", linewidth=0.5)
    axes[1].legend()

    fig.suptitle(f"Actual vs Estimated O3 — {model_name}", fontsize=13)
    fig.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close(fig)

plot_actual_vs_pred_dual(y_train_raw, ytr_ffnn_pred, y_test_raw, yte_ffnn_pred,
                         f"PSO-FFNN (H={HIDDEN_NEURONS}, act={ACTIVATION})",
                         "figs/actual_estimated_ffnn.png")

plot_actual_vs_pred_dual(y_train_raw, ytr_elm_pred, y_test_raw, yte_elm_pred,
                         f"PSO-ELM (H={HIDDEN_NEURONS}, act={ACTIVATION})",
                         "figs/actual_estimated_elm.png")

In [80]:
def plot_pso_convergence_twofig(hist_ffnn, hist_elm,
                                path_ffnn="figs/pso_convergence_ffnn.png",
                                path_elm="figs/pso_convergence_elm.png",
                                metric_label="RMSE (scaled y)"):
    import os, numpy as np, matplotlib.pyplot as plt
    from matplotlib.ticker import MaxNLocator

    os.makedirs("figs", exist_ok=True)

    # Flatten & common y-limits (start at 0)
    h1 = np.asarray(hist_ffnn, dtype=float).ravel()
    h2 = np.asarray(hist_elm,  dtype=float).ravel()
    ymax = float(np.nanmax([h1.max(), h2.max()]))
    pad  = 0.02 * (ymax + 1e-12)
    upper = np.ceil((ymax + pad) * 10) / 10.0
    lo, hi = 0.0, upper

    # ---- Figure 1: FFNN ----
    fig1, ax1 = plt.subplots(figsize=(7, 4))
    ax1.plot(h1, lw=1.8)
    ax1.set_title("PSO Convergence — FFNN")
    ax1.set_xlabel("Iteration")
    ax1.set_ylabel(f"Best {metric_label}")
    ax1.set_ylim(lo, hi)
    ax1.yaxis.set_major_locator(MaxNLocator(nbins=6))
    ax1.grid(True, linestyle="--", linewidth=0.5)
    fig1.tight_layout()
    fig1.savefig(path_ffnn, dpi=150, bbox_inches="tight")
    plt.close(fig1)

    # ---- Figure 2: ELM ----
    fig2, ax2 = plt.subplots(figsize=(7, 4))
    ax2.plot(h2, lw=1.8)
    ax2.set_title("PSO Convergence — ELM")
    ax2.set_xlabel("Iteration")
    ax2.set_ylabel(f"Best {metric_label}")
    ax2.set_ylim(lo, hi)  # same limits as FFNN
    ax2.yaxis.set_major_locator(MaxNLocator(nbins=6))
    ax2.grid(True, linestyle="--", linewidth=0.5)
    fig2.tight_layout()
    fig2.savefig(path_elm, dpi=150, bbox_inches="tight")
    plt.close(fig2)

    print(f"Saved: {path_ffnn}")
    print(f"Saved: {path_elm}")

plot_pso_convergence_twofig(
    ffnn_hist, elm_hist,
    path_ffnn="figs/pso_convergence_ffnn.png",
    path_elm="figs/pso_convergence_elm.png",
    metric_label="RMSE (scaled y)"  # change to "MSE (scaled y)" if you switched fitness
)


Saved: figs/pso_convergence_ffnn.png
Saved: figs/pso_convergence_elm.png


In [81]:
import pandas as pd

metrics_df = pd.DataFrame({
    "FFNN-Train": ffnn_train,
    "FFNN-Test":  ffnn_test,
    "ELM-Train":  elm_train,
    "ELM-Test":   elm_test,
}).T  # models become rows

# Optional: re-order columns and round
metrics_df = metrics_df[["MAE", "RMSE", "R2", "MAPE_%"]].round(4)

metrics_df

,MAE,RMSE,R2,MAPE_%
FFNN-Train,4.0300,5.1180,0.7619,19.0687
FFNN-Test,4.6816,5.8831,0.6761,20.8612
ELM-Train,2.6786,3.3976,0.8951,12.0926
ELM-Test,4.2907,5.5859,0.7080,18.4416


In [82]:
import numpy as np
import matplotlib.pyplot as plt
import os

os.makedirs("figs", exist_ok=True)

def _as1d(a):
    return np.asarray(a, dtype=float).ravel()

def plot_curves_train_test_both_models(ytr, ytr_ffnn, ytr_elm,
                                       yte, yte_ffnn, yte_elm,
                                       path="figs/curves_train_test_ffnn_elm.png"):
    # flatten everything
    ytr       = _as1d(ytr)
    ytr_ffnn  = _as1d(ytr_ffnn)
    ytr_elm   = _as1d(ytr_elm)
    yte       = _as1d(yte)
    yte_ffnn  = _as1d(yte_ffnn)
    yte_elm   = _as1d(yte_elm)

    # shared y-limits
    all_vals = np.concatenate([ytr, ytr_ffnn, ytr_elm, yte, yte_ffnn, yte_elm])
    y_min, y_max = float(all_vals.min()), float(all_vals.max())
    pad = 0.02 * (y_max - y_min + 1e-9)
    lo, hi = y_min - pad, y_max + pad

    fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharey=True)

    # (0,0) FFNN — Train
    axes[0,0].plot(ytr,      label="Actual")
    axes[0,0].plot(ytr_ffnn, label="Estimated", alpha=0.9)
    axes[0,0].set_title("FFNN — Train")
    axes[0,0].set_ylabel("Target")
    axes[0,0].grid(True, linestyle="--", linewidth=0.5)
    axes[0,0].legend()
    axes[0,0].set_ylim(lo, hi)

    # (0,1) FFNN — Test
    axes[0,1].plot(yte,      label="Actual")
    axes[0,1].plot(yte_ffnn, label="Estimated", alpha=0.9)
    axes[0,1].set_title("FFNN — Test")
    axes[0,1].grid(True, linestyle="--", linewidth=0.5)
    axes[0,1].legend()
    axes[0,1].set_ylim(lo, hi)

    # (1,0) ELM — Train
    axes[1,0].plot(ytr,     label="Actual")
    axes[1,0].plot(ytr_elm, label="Estimated", alpha=0.9)
    axes[1,0].set_title("ELM — Train")
    axes[1,0].set_xlabel("Samples (order in file)")
    axes[1,0].set_ylabel("Target")
    axes[1,0].grid(True, linestyle="--", linewidth=0.5)
    axes[1,0].legend()
    axes[1,0].set_ylim(lo, hi)

    # (1,1) ELM — Test
    axes[1,1].plot(yte,     label="Actual")
    axes[1,1].plot(yte_elm, label="Estimated", alpha=0.9)
    axes[1,1].set_title("ELM — Test")
    axes[1,1].set_xlabel("Samples (order in file)")
    axes[1,1].grid(True, linestyle="--", linewidth=0.5)
    axes[1,1].legend()
    axes[1,1].set_ylim(lo, hi)

    fig.suptitle("Actual vs Estimated — Train & Test (FFNN and ELM)", fontsize=13)
    fig.tight_layout()
    plt.savefig(path, dpi=150)
    plt.close(fig)
    print(f"Saved: {path}")


In [83]:
plot_curves_train_test_both_models(
    ytr=y_train_raw,
    ytr_ffnn=ytr_ffnn_pred,
    ytr_elm=ytr_elm_pred,
    yte=y_test_raw,
    yte_ffnn=yte_ffnn_pred,
    yte_elm=yte_elm_pred,
    path="figs/curves_train_test_ffnn_elm.png"
)

Saved: figs/curves_train_test_ffnn_elm.png


In [84]:
plot_actual_vs_estimated_grid(
    ytr=y_train_raw, ytr_ffnn=ytr_ffnn_pred, ytr_elm=ytr_elm_pred,
    yte=y_test_raw,  yte_ffnn=yte_ffnn_pred, yte_elm=yte_elm_pred,
    path="figs/actual_vs_estimated_grid.png"
)

Saved: figs/actual_vs_estimated_grid.png
